In [1]:
import pandas as pd
path_blast = 'data/results/mites_clustered.blast.csv'
path_output = 'data/results/mites_clustered.blast.filtered.csv'

In [ ]:
#TEs
params = {}
params['LTR'] = {'min_len':650,'max_len':False,'min_distance':5,'max_q':1.4,'min_q':0.6,'min_pident':60,'min_qcov':80,'file':''}
params['LINE'] = {'min_len':1500,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':60,'min_qcov':80,'file':''}
params['SINE'] = {'min_len':150,'max_len':800,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':''}
params['TIR'] = {'min_len':700,'max_len':False,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':''}
params['MITE'] = {'min_len':50,'max_len':800,'min_distance':5,'max_q':1.15,'min_q':0.85,'min_pident':90,'min_qcov':90,'file':''}
params['Hel'] = {'min_len':2000,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':60,'min_qcov':80,'file':''}

te_type = 'LTR'

In [ ]:
#read blast output
df = pd.read_csv(dir_blast + params[te_type]['file'], sep='\t', header=None)
df.columns = ['qseqid,''sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df_blast.index))

In [ ]:
#filter by length
if(params[te_type]['min_len']):
    df = df[(df.qlen > params[te_type]['min_len'])]
    
if(params[te_type]['max_len']):
    df = df[(df.qlen < params[te_type]['max_len'])]

print('Min and max len: ' + str(len(df.index)))

In [ ]:
#filter by query / subject length treshold
df_blast = df_blast[((df_blast.length / df_blast.qlen) >= params[te_type]['min_q']) &
   ((df_blast.length / df_blast.slen) >= params[te_type]['min_q']) &
   ((df_blast.length / df_blast.qlen) <= params[te_type]['max_q']) &
   ((df_blast.length / df_blast.slen) <= params[te_type]['max_q'])]
print('treshold:',len(df_blast.index))

In [ ]:
#filter by pident and qcov
df = df[(df.pident > params[te_type]['min_pident']) & (df.qcovs > params[te_type]['min_qcov'])]
print('Min pident and min qcov: ' + str(len(df.index)))

In [ ]:
#filter overlapped 

# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]

#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']

# filtering
rows = []
discard = []
for index, row in df.iterrows():
    if index in discard:
        continue
    df = dfs[row.sseqid]
    res = df[(abs(df.sstart - row.sstart) <= params[te_type]['min_distance']) | (abs(df.send - row.send) <= params[te_type]['min_distance'])]
    if len(res.index) > 1:
        print(res)
        break
        added = False
        for i2, r2 in res.iterrows():
            if not added:
                rows.append(r2)
                added = True
            else:
                discard.append(r2.newindex)
    else:
        rows.append(row)

df = pd.DataFrame(rows)
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1).drop('newindex',axis=1).drop('q',axis=1)
df.sort_values(['sseqid', 'sstart'], inplace=True)
print('Non overlapped: ' + str(len(df.index)))

In [ ]:
df.to_csv()